### Looking into using a model monitoring package (evidently) to look at the `penguins` dataset from `seaborn`

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from jmspack.utils import JmsColors

from evidently.dashboard import Dashboard
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.dashboard.tabs import (DataDriftTab,
                                        DataQualityTab,
                                        ProbClassificationPerformanceTab)
from evidently.options import DataDriftOptions
from evidently.options import ColorOptions
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = sns.load_dataset("penguins").dropna()
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


In [3]:
target="sex"
target_names=df[target].unique().tolist()
numerical_feature_names=df.drop(target, axis=1).select_dtypes(float).columns.tolist()
categorical_feature_names=df.drop(numerical_feature_names + [target], axis=1).columns.tolist()
target_names,numerical_feature_names,categorical_feature_names

(['Male', 'Female'],
 ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
 ['species', 'island'])

In [4]:
df[categorical_feature_names] = df[categorical_feature_names].astype("category").apply(lambda s: s.cat.codes)

In [5]:
X_train = df[numerical_feature_names + categorical_feature_names].sample(frac=0.8)
y_train = df.loc[X_train.index, target]
X_test = df[numerical_feature_names + categorical_feature_names].drop(X_train.index.tolist())
y_test = df.loc[X_test.index, target]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((266, 6), (266,), (67, 6), (67,))

In [6]:
model = RandomForestClassifier(random_state=42)
_ = model.fit(X=X_train, y=y_train)

In [7]:
pred_out, pred_in = target_names 

# Create probabilities dfs for reference and test dfs and merge with original dfs i.e,
# Outdoors | Indoors | Prediction
probas_reference = (pd.DataFrame(data = model.predict_proba(X_train), 
                                    columns = target_names)
                        .assign(Prediction = lambda x: np.where(x[pred_in] > x[pred_out], pred_in, pred_out))
                            )
merged_reference = pd.concat([X_train.reset_index(drop=True), probas_reference.reset_index(drop=True)], axis=1)

probas_test = (pd.DataFrame(data = model.predict_proba(X_test), 
                                    columns = target_names)
                        .assign(Prediction = lambda x: np.where(x[pred_in] > x[pred_out], pred_in, pred_out))
                            )
merged_test = pd.concat([X_test.reset_index(drop=True), probas_reference.reset_index(drop=True)], axis=1)

In [8]:
# Define the color scheme that is going to be used in the Reports
color_scheme = ColorOptions(primary_color = JmsColors.PURPLE,
                            secondary_color = JmsColors.YELLOW,
                            fill_color =  JmsColors.GREENYELLOW, 
                            zero_line_color = JmsColors.DARKGREY,
                            color_sequence = JmsColors.to_list()
                            )

# Define the column mapping, essential step to generate the reports.
indoor_outdoor_col_map = ColumnMapping(prediction = target_names,
                                        target = "Prediction",
                                        target_names = target_names,
                                        numerical_features = numerical_feature_names,
                                        categorical_features = categorical_feature_names,
                                        pos_label = target_names[1])

In [9]:
# Other configurations

bin_size = 25
# specify a bin size of 25 for the columns that are only included in the model
# NOT the columns that also evidently produce (target columns)
specs_dict = dict.fromkeys(numerical_feature_names, bin_size)
options = DataDriftOptions(nbinsx=specs_dict, drift_share=0.6)

In [10]:
# Initiate the Evidently Dashboards
summary_dashboard = Dashboard(tabs=[DataDriftTab(include_widgets=['Data Drift']), 
                                    DataQualityTab(include_widgets=['Data Summary', 'Features']),
                                    ProbClassificationPerformanceTab(include_widgets=['Reference: Class Representation',
                                                                                        'Current: Class Representation',
                                                                                        'Reference: Class Separation Quality',
                                                                                        'Current: Class Separation Quality'])],
                    options=[color_scheme, options])

# Run and save all calculations
summary_dashboard.calculate(reference_data = merged_reference, 
                            current_data = merged_test,
                            column_mapping = indoor_outdoor_col_map)

In [11]:
filename = "penguins_data_drift_report_evidently.html"
export_data=False
if export_data:
    summary_dashboard.save(filename)